In [61]:
## In class 2021-09-20

In [62]:
## Q1 Load the data file to your S3 bucket. Using the pandas library, read the csv data file and create a data-frame 
## called heart

import pandas as pd
import boto3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import accuracy_score

## Defining the S3 bucket
s3 = boto3.resource('s3')
bucket_name = 'bonnieh-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [63]:
## Q2 Remove observations with missing values.

heart = heart.dropna()
heart

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4231,1,58,3.0,0,0.0,0.0,0,1,0,187.0,141.0,81.0,24.96,80.0,81.0,0
4232,1,68,1.0,0,0.0,0.0,0,1,0,176.0,168.0,97.0,23.14,60.0,79.0,1
4233,1,50,1.0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0,1
4234,1,51,3.0,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0,0


In [64]:
## Q3 Using age, currentSmoker, totChol, BMI, and heartRate as the predictor variables, and
## TenYearCHD is the target variable, split the data into two data-frames: train (80%) and test (20%).

## defining the variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## splitting the data

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size =0.20)
X_train

,age,currentSmoker,totChol,BMI,heartRate
514,56,1,278.0,22.67,85.0
2856,50,1,320.0,25.45,90.0
2158,40,1,162.0,24.12,70.0
1489,55,0,230.0,28.27,72.0
1093,43,1,263.0,25.68,70.0
...,...,...,...,...,...
666,57,0,287.0,26.24,65.0
879,49,0,264.0,25.16,76.0
566,60,0,232.0,29.23,69.0
3940,47,1,260.0,26.58,90.0


In [65]:
## Q4 Normalize the inputs variables of the train and test datasets using the L2 normalization.
## That is, for each input variable subtract the mean of that variable, then divide by the L2-norm of that
## variable.

def l2_normalization(X):
    x_mean = np.mean(X)
    l2 = np.sqrt(sum(X**2))
    
    return(X-x_mean) /l2

X_train = X_train.apply(l2_normalization, axis = 1)
X_test = X_test.apply(l2_normalization, axis = 1)


In [66]:
## Q5  Using the train data-frame, build a logistic regression model in which: age, currentSmoker,
## totChol, BMI, and heartRate are the predictor variables, and TenYearCHD is the target variable. Using
## this model, predict the likelihood of risk of coronary disease of the patients in the test data-frame. Using
## 25% as cutoff value, report the accuracy.

## building the logistric regression model
md1 = LogisticRegression().fit(X_train, Y_train)

## predicting on the test dataset
pred1 = md1.predict_proba(X_test)[:,1]

## setting the threshold at 25%
pred1 = np.where(pred1 < 0.25, 0, 1)
pred1

## computing the accuracy of the model
accuracy_score(Y_test, pred1)

0.8415300546448088

In [67]:
## Q6 Using the train data-frame, build a perceptron model in which: age, currentSmoker,
## totChol, BMI, and heartRate are the predictor variables, and TenYearCHD is the target variable. Use
## penalty = ’l2’ and alpha = 0.01. Using this model, predict the risk of coronary disease of the
## patients in the test data-frame. Report the accuracy.

## building the perceptron model
md2 = Perceptron(penalty='l2', alpha = 0.01).fit(X_train, Y_train)

md2.coef_

array([[ 0.32909337, -0.35745272,  0.21572706, -0.18402368, -0.00334403]])

In [68]:
## predicting on the test dataset
pred2 = md2.predict(X_test)

## Computing the accuracy
accuracy_score(Y_test, pred2)

0.15846994535519127

In [ ]:
## Q7 Using the results from parts (5) and (6), what model would you use to predict TenYearCHD?

## Most runs give identical accuracy scores for the Logistic Regression and the Perceptron model. The Logistic
## regression model didn't vary much in accuracy. The Perceptron model was mostly good, but the accuracy sometimes 
## dropped to 15% (compared to 85%). So I would use the Logistic Regression model--it is more accurate based on
## multiple runs.